# ACME Intelligence Slack Bot - Quick Start

**Interactive testing for Snowflake Cortex Agents**

This notebook lets you:
1. Test your Snowflake agents interactively
2. See real streaming responses
3. Experiment with different questions
4. Understand the API behavior before deploying to Slack


In [ ]:
# Simple imports - just what we need
import requests
import json
import re
from typing import Optional


In [ ]:
# Configuration
import os

# Set these via environment variables or directly here
ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT", "trb65519")
PAT_TOKEN = os.getenv("SNOWFLAKE_PAT", "your-pat-token-here")

# Available agents in your Snowflake Intelligence platform
AGENTS = {
    "intelligence": "ACME_INTELLIGENCE_AGENT",       # General business intelligence
    "contracts": "ACME_CONTRACTS_AGENT",             # Contract analysis
    "perf": "DATA_ENGINEER_ASSISTANT",               # Query performance
}


## Core Function

One function to call any Snowflake Cortex Agent.


In [ ]:
def ask_agent(question: str, agent: str = "intelligence") -> str:
    """
    Call the Snowflake Agent Object API with a natural language question.
    
    Args:
        question: Natural language question
        agent: Which agent to use (intelligence, contracts, perf)
        
    Returns:
        The agent's response text
    """
    agent_name = AGENTS.get(agent, AGENTS["intelligence"])
    
    # Construct Agent Object API endpoint
    url = f"https://{ACCOUNT}.snowflakecomputing.com/api/v2/databases/SNOWFLAKE_INTELLIGENCE/schemas/AGENTS/agents/{agent_name}:run"
    
    # Simple request payload
    payload = {
        "messages": [{
            "role": "user",
            "content": [{"type": "text", "text": question}]
        }]
    }
    
    # Authentication with PAT token
    headers = {
        "Authorization": f"Bearer {PAT_TOKEN}",
        "X-Snowflake-Authorization-Token-Type": "PROGRAMMATIC_ACCESS_TOKEN",
        "Content-Type": "application/json"
    }
    
    # Make the API call with streaming enabled
    response = requests.post(url, json=payload, headers=headers, stream=True, timeout=60)
    response.raise_for_status()
    
    # Parse Server-Sent Events stream and extract text
    text_parts = []
    for line in response.iter_lines(decode_unicode=True):
        if line.startswith('data: '):
            try:
                data = json.loads(line[6:])  # Remove 'data: ' prefix
                if 'text' in data:
                    text_parts.append(data['text'])
            except: 
                pass
    
    return "".join(text_parts) if text_parts else "No response"


## Test Different Question Types

The ACME Intelligence Agent can handle business intelligence, contracts, and operational questions.


In [ ]:
# Business intelligence questions
answer = ask_agent("How many customers do we have?")
print(answer)


In [ ]:
# Contract questions
answer = ask_agent("Which contracts are at risk?", agent="contracts")
print(answer)


In [ ]:
# Performance questions
answer = ask_agent("What are the slowest queries today?", agent="perf")
print(answer)


## Add Smart Routing (Optional)

For convenience, you can add automatic agent selection based on question keywords.


In [ ]:
def auto_route(question: str) -> str:
    """
    Automatically select the best agent based on question keywords.
    
    Note: This is optional - the main intelligence agent can handle most questions.
    """
    q = question.lower()
    
    # Keyword-based routing
    if any(word in q for word in ['contract', 'churn', 'risk', 'renewal']):
        return "contracts"
    elif any(word in q for word in ['slow', 'query', 'performance', 'warehouse']):
        return "perf"
    else:
        return "intelligence"

def ask(question: str) -> str:
    """
    Ask a question with automatic agent selection.
    
    Simpler interface that routes to the appropriate agent automatically.
    """
    agent = auto_route(question)
    print(f"🎯 Routing to: {AGENTS[agent]}")
    return ask_agent(question, agent)


In [ ]:
# Test with automatic routing
print("Question 1:")
print(ask("How many customers do we have?"))

print("\nQuestion 2:")
print(ask("Which contracts are expiring next month?"))

print("\nQuestion 3:")
print(ask("Show me slow queries from yesterday"))


## Next Steps

Now that you've tested the core functionality:

1. **Experiment with questions** - Try different question types
2. **Review responses** - See how the agent handles different scenarios
3. **Extend as needed** - Add features based on your requirements
4. **Deploy to Slack** - Connect to your workspace when ready

See `README.md` for Slack deployment instructions and production considerations.
